In [1]:
%load_ext lab_black

In [2]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
from azureml.core import Environment, Workspace

# Initialize Azure ML Workspace
ws = Workspace.from_config()

# Create a Python environment for the experiment
python_env = Environment.from_pip_requirements("pipeline_env", "requirements.txt")

In [8]:
# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()
pipeline_run_config.environment = python_env

from azureml.core.compute import ComputeTarget

# Define the name of your existing compute target
existing_compute_target_name = "matthewhanauer993"

# Retrieve the existing compute target
compute_target = ComputeTarget(workspace=ws, name=existing_compute_target_name)

# Validate that the compute target exists
if compute_target is not None:
    print(f"Using existing compute target: {existing_compute_target_name}")
else:
    print(f"No compute target found with name {existing_compute_target_name}")
    exit(1)  # Exit the script if the compute target doesn't exist

# Step to manage Docker services
docker_script_step = PythonScriptStep(
    name="Manage Docker Services",
    script_name="manage_docker.py",
    arguments=["start"],  # or ["stop"] to stop services
    source_directory="./",
    runconfig=pipeline_run_config,
    compute_target=compute_target,
    allow_reuse=False,
)

# Create and submit the pipeline
pipeline = Pipeline(workspace=ws, steps=[docker_script_step])
experiment_name = 'My_Docker_Pipeline'
experiment = Experiment(workspace=ws, name=experiment_name)
pipeline_run = experiment.submit(pipeline)

published_pipeline = pipeline.publish(
    name="My_Published_Pipeline",
    description="Published pipeline to run the script every minute.",
)

pipeline_id = published_pipeline.id

Using existing compute target: matthewhanauer993
Created step Manage Docker Services [c6c67963][7de3e201-860c-4ee2-94bb-8146437020c3], (This step will run and generate new outputs)
Submitted PipelineRun 1571c6bf-183b-4b03-bff4-8dac0f42e464
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1571c6bf-183b-4b03-bff4-8dac0f42e464?wsid=/subscriptions/d3ade6e0-a7be-4323-ace4-02b7b4da2451/resourcegroups/ml_modeling/workspaces/mhanauer_1&tid=a93eafc8-c064-4a88-bc01-47e93068b42b


In [10]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Define the pipeline schedule
recurrence = ScheduleRecurrence(frequency="Minute", interval=1)  # Runs every minute
schedule = Schedule.create(
    ws,
    name="My_Schedule",
    description="Runs pipeline every minute",
    pipeline_id=pipeline_id,
    experiment_name=experiment_name,
    recurrence=recurrence,
)

In [ ]:
from azureml.core import Workspace
from azureml.pipeline.core import Schedule

# Initialize Azure ML Workspace
ws = Workspace.from_config()

# Your schedule name
schedule_name = "My_Schedule"

# List all schedules in the workspace
schedules = Schedule.list(ws)

# Filter and disable the schedule with the given name
for schedule in schedules:
    if schedule.name == schedule_name:
        print(f"Disabling schedule {schedule.name}")
        schedule.disable()